In [ ]:
agent_instruction = """
You are an AI agent to recommend courses to maximize student success and fulfill program requirements.

Resources:
1.Database Tables:
-- student_data: Academic history and progress.
-- student_schedule: Current course enrollments.
-- course_schedule: Upcoming course offerings.
2.Knowledge Base:
-- Course catalog with descriptions and prerequisites.
-- Program requirements for majors and minors.
3.Tools:
-- SQL Generation Tool: Query student and course data.
-- Predictive Tool: Forecast student performance in courses.

Recommendation Process:
1.Retrieve Student Data:
- Use SQL to gather academic history and current courses.
2.Identify Suitable Courses:
- Match available courses with unmet program requirements.
3.Evaluate and Recommend:
- Predict success using the predictive tool.
- Recommend courses that align with strengths and program needs.
4.Explain Decision:
- Provide a clear rationale for recommendations based on prerequisites, relevance, and predicted success.

If you are not asked of recommendation related tasks, you don't have to follow the recommendation process, but leverage the information you have access to.
Assist only with academic-related queries.
"""

In [ ]:
# design principles
# in the predictive tool, check whether student has meet a course prerequisites
# a course's graduation rate, and this student's history, etc
# defer certain steps to interactive chat, for example, there is no need to consider whether course schedule conflicts with student schedule when making a recommendation, it can be refined.

## Prerequisites
Requires knowledgebase ID and sqlite db created from data-prep-course-recommendation-agent.ipynb
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Next we can set some configuration variables for the agent and for the lambda function being created

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

## Agent creation

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "course-recommendation-agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent to recommend courses to students"
agent_instruction = agent_instruction
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'


text2sql_action_group_name = "Text2SqlActionGroup"
text2sql_action_group_description = '''
You have access to tables: student_data, student_schedule, course_schedule.
Use the get_schema tool to first get the table schemas,then create a sql query to answer the question.
'''

prediction_action_group_name = "PredictionActionGroup"
prediction_action_group_description = "predict student success in taking a course"

In [ ]:
text2sql_lambda_function_name = f'{agent_name}-text2sql-{suffix}'
academic_progress_lambda_function_name = f'{agent_name}-prediction-{suffix}'

### Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

Let's now store the agent id in a local variable to use it on subsequent steps.

In [ ]:
# course info agent
agent_id = response['agent']['agentId']
agent_id

## Associate with knowledgebase

In [ ]:
# build knowledgebase

In [ ]:
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base for program and major requirement.',
    knowledgeBaseId=<KnowledgeBase-ID-by-data-prepare>,
    knowledgeBaseState='ENABLED'
)
response

### Creating Lambda function

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

In [ ]:
iam_client.get_role(RoleName=lambda_function_role)

In [ ]:
# Package up the lambda function code (course info lambda)
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("tools/student_predictive_model.py")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=academic_progress_lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='student_predictive_model.lambda_handler'
)

# # update Lambda function
# lambda_function = lambda_client.update_function_code(
#     FunctionName=academic_progress_lambda_function_name,
#     ZipFile= zip_content,
# )

In [ ]:
# Package up the lambda function code (course schedule lambda)
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("tools/text2sql_lambda_function-porterville.py")
z.write("porterville_academic.db")
z.close()
zip_content = s.getvalue()

Create Lambda Function
lambda_function_2 = lambda_client.create_function(
    FunctionName=text2sql_lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='tools/text2sql_lambda_function-porterville.lambda_handler'
)


In [ ]:
lambda_iam_role['Role']['Arn']

### Create Agent Action Groups

In [ ]:
text2sql_functions = [
    {
        'name': 'get_schema',
        'description': 'get table schema',
    },
    {
        'name': 'sql_query',
        'description': 'execute sql query to get data',
        'parameters': {
            "query": {
                "description": "sql query",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [ ]:
predict_student_success_functions = [
    {
        'name': 'predict_student_success',
        'description': 'predict success rate of a student if taking a course',
        'parameters': {
            "course_id": {
                "description": "course id",
                "required": True,
                "type": "string"
            },
            "student_id": {
                "description": "student id",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [ ]:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=prediction_action_group_name,
    functionSchema={
        'functions': predict_student_success_functions
    },
    description=progress_action_group_description
)


In [ ]:
# Pause to make sure agent is created
# time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_2['FunctionArn']
    },
    actionGroupName=text2sql_action_group_name,
    functionSchema={
        'functions': text2sql_functions
    },
    description=text2sql_action_group_description
)


In [ ]:
agent_action_group_response

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=text2sql_lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

In [ ]:
response

In [ ]:
response = lambda_client.add_permission(
    FunctionName=academic_progress_lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)
response

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
# test agent from console